# Voting

In [27]:
#1

# Importing the libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

# Importing the dataset
dataset = pd.read_csv('CVD-dataset-plin-0-4-6-7-9.csv')
# dataset = pd.read_csv('CVD dataset-plin-4-7-9.csv') #διαβάζω το dataset που δεν περιλαμβανει τα features 7 και 9
data=dataset.iloc[:, :].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 11].values

rows = len(data)    # 3 rows in your example
cols = len(data[0])
print(rows)
print(cols)




560
12


In [28]:
#2

dataNoCvd = data[~(data[:,11] > 0.0)] #χωρίζω τα data σε 2 πίνακες, ο ένας με τα cvd ο άλλος με τα όχι cvd
dataCvd = data[~(data[:,11] < 1.0)]
rowsNoCvd = len(dataNoCvd)    
rowsCvd = len(dataCvd)
print(rowsNoCvd)
print(rowsCvd)

519
41


## Training Testing Validation sets

Τα instances με cvd είναι 41 και χωρίς cvd είναι 519, συνολικά είναι δηλαδή 560. Άρα στο 10-fold cross validation το test set θα περιλαμβάνει 56 instances και τα υπόλοιπα 504 θα διατεθούν για το training. Όμως επειδή το 10% του training θα δωθεί στο validation set αυτό σημαίνει ότι τελικά 50 instances θα ανήκουν στο validation set και με τα υπόλοιπα 454 θα γίνεται το training.

### Testing
41//10=4 και υπόλοιπο 1
519//10=51 και υπόλοιπο 9 
Γιαυτό τα πρώτα 9 test set θα πάρουν από ένα extra NoCvd instance άρα θα έχουν NoCvd:52 Cvd:4
το 10ο test set θα πάρει το  extra Cvd instance άρα θα έχει NoCvd:51 Cvd:5

### Validation
Το validation set θα έχει όπως είπαμε 50 instances και σταθερά NoCvd:46 Cvd:4
Για να φτιάξουμε το validation set για κάθε fold θα παίρνουμε κάθε φορά τα επόμενα 46 instances από αυτά που πήραμε για το test set από τον πίνακα που θα περιέχει όλα τα instances χωρίς cvd και τα επόμενα 4 instances από αυτά που πήραμε για το test set από τον πίνακα που θα περιέχει όλα τα instances με cvd. Μόνο στην περίπτωση του 10ου fold θα πάρουμε τα 46 και τα 4 instances που προηγούνται των instances που πήραμε για το test set και αυτό γιατί δεν θα υπάρχουν επόμενα instances για να τα πάρουμε. Το ακόλουθο σχήμα δείχνει πώς γίνεται ο χωρισμός στην περίπτωση των 1,2,9 και 10 fold.

### Training 
Όσα instances περισσεύουν από το test και validation set. Άρα ο συνολικός του αριθμός θα είναι 454 και στα πρώτα 9 folds θα έχει NoCvd:421 Cvd:33 και στο 10ο fold θα έχει NoCvd:422 και Cvd:32

<!-- <img src="fold-1-2.png"> -->
<img src="fold-1-2-site.svg">
<img src="fold-9-10-site.svg">


In [29]:
def find_NoCvd_limitsArray(i,DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue):
    if i>0 :
        DownNoCvd = UpNoCvd
        UpNoCvd= fold_NoCvd_total+UpNoCvd
    if i < fold_NoCvd_residue :
        UpNoCvd+= 1
    return   DownNoCvd, UpNoCvd   
 
def find_Cvd_limitsArray(i,DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv):
    if i>0 :
        DownCvd = UpCvd
        UpCvd= fold_Cvd_total+UpCvd
    if i >= cv - fold_Cvd_residue :
         UpCvd+= 1     
    return   DownCvd, UpCvd  

def find_testValSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1=dataNoCvd[DownNoCvd:UpNoCvd,:]
    temp2=dataCvd[DownCvd:UpCvd,:]
    temp3=np.concatenate((temp1, temp2))
    return temp3

def find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.concatenate((temp1, temp2))
    return temp1, temp2, temp3

def find_trainSubset1(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    print("len 1-2:",len(temp1),len(temp2))
    temp3 = np.delete(temp1, slice(0, 46), axis=0)
    temp4 = np.delete(temp2, slice(0, 4), axis=0)
    print("len 3-4:",len(temp3),len(temp4))
    temp5 = np.concatenate((temp3, temp4))
    print(len(temp5))
    return temp3, temp4, temp5

In [51]:
def create_train_test_sets(cv,dataNoCvd,dataCvd,val_ratio):
    # cv= αριθμός των folds, κάνω 10-fold cross validation γιαυτό το έχω θέσει 10
    # val_ratio= το ποσοστό του training set που θα γίνει validation set, το έβαλα 10%
    test_total = []
    train_total = []
    train_total_Cvd = []
    train_total_NoCvd = []
    validation_total = []
    
    #στόχος είναι να χωρίσουμε τα δεδομένα σε 10 folds με ποσοστό instances με cvd περιπου 7%, το 1 fold θα                                
    #χρησιμοποιείται για test και τα άλλα 9 θα ενωνονται και θα χρησιμοποιούνται για train-validation
    
    rowsNoCvd = len(dataNoCvd)    # βρίσκω τον αριθμό των instances χωρίς cvd
    rowsCvd = len(dataCvd)        # βρίσκω τον αριθμό των instances με cvd

    #fold_Cvd_total = αριθμός των instances με cvd που θα μπουν  σίγουρα στο 1 fold = 4
    fold_Cvd_total = rowsCvd//cv 
    #fold_Cvd_residue = το υπόλοιπο των instances που πρέπει να το μοιράσουμε στα 10 folds, δηλαδη κάποια 
    #θα έχουν ένα instance με cvd παραπάνω = 1
    fold_Cvd_residue= rowsCvd%cv
    print("fold_Cvd_total  :",fold_Cvd_total," fold_Cvd_residue  :",fold_Cvd_residue)

    #fold_NoCvd_total = αριθμός των instances χωρίς cvd που θα μπουν  σίγουρα στο 1 fold = 51
    fold_NoCvd_total = rowsNoCvd//cv
    #fold_NoCvd_residue = το υπόλοιπο των instances που πρέπει να το μοιράσουμε στα 10 folds, δηλαδη κάποια 
    #θα έχουν ένα instance χωρίς cvd παραπάνω = 9
    fold_NoCvd_residue= rowsNoCvd%cv
    print("fold_NoCvd_total:",fold_NoCvd_total,"fold_NoCvd_residue:",fold_NoCvd_residue)
    
    #Cvd_val= αριθμός instances με cvd που θα μπει στο validation set,το train set αποτελείται από 504 instances  
    #από αυτά το 7% είναι cvd, εμείς από το 7% θα πάρουμε το 10% για το validatio set
    Cvd_val = round(504*val_ratio*(rowsCvd/len(data)))
    #504*val_ratio= ο συνολικός αριθμός των instances  με του validation set, άρα αν από αυτόν αφαιρέσουμε 
    #το Cvd_val θα πάρουμε των αριθμό των instances χωρίς cvd που θα μπουν στο validation set
    noCvd_val = round(504*val_ratio)-Cvd_val
    
    #dataNoCvd= [0 1 2 .... 517 518] αυτοί είναι οι δείκτες του πίνακα με όλα τα NoCvd instances, τα  DownNoCvd και
    # UpNoCvd θα είναι οι δείκτες στους οποίους θα γίνεται το χώρισμα του πίνακα dataNoCvd κάθε φορά προκειμένου
    # να πάρουμε ένα fold, DownNoCvd=κάτω όριο, UpNoCvd= πάνω όριο, ξεκινάμε με DownNoCvd=0(από την αρχή του πίνακα)
    # UpNoCvd = fold_NoCvd_total(αριθμός NoCvd instances που θα μπουν σίγουρα στο fold)
    #
    #dataCvd= [0 1 2.. 39 40] δείκτες του πίνακα dataCvd, με την ίδια λογική με πριν DownCvd=0=κάτω όριο-δείκτης
    # UpCvd= άνω όριο-δείκτης=fold_Cvd_total(αριθμός Cvd instances που θα μπουν σίγουρα στο fold)
    DownNoCvd=0
    DownCvd=0
    UpNoCvd = fold_NoCvd_total
    UpCvd = fold_Cvd_total
    #τα όρια αυτά θα αλλάξουν 10 φορές, όσα είναι και τα fold 
    
    for i in range(cv):
        X_test = []
        y_test = []
        X_train = []
        y_train = []
        X_val = []
        y_val = []
        
        #find_NoCvd_limitsArray = βρίσκει τα όρια του dataNoCvd πίνακα για το i fold
        DownNoCvd, UpNoCvd = find_NoCvd_limitsArray(i, DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue)
        #find_Cvd_limitsArray = βρίσκει τα όρια του dataCvd πίνακα για το i fold
        DownCvd, UpCvd = find_Cvd_limitsArray(i, DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv)

        #find_testValSubset = επιστρέφει τον πίκακα test για το i-fold
        testSubset_total = find_testValSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)        
        
        if i!=9:
        #όταν δεν είμαστε στο 10ο fold(ξεκινάμε από 0) παίρνουμε τα επόμενα 56 και 4 instaces για το validation set
            validationSubset_total = find_testValSubset(UpNoCvd,UpNoCvd+noCvd_val,UpCvd,UpCvd+Cvd_val)
            trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd,UpNoCvd+noCvd_val,DownCvd,UpCvd+Cvd_val)
        else:
        #όταν είμαστε στο 10ο fold παίρνουμε τα προηγούμενα 56 και 4 instaces για το validation set
#         DownNoCvd-noCvd_val,DownNoCvd,DownCvd-Cvd_val,DownCvd
            validationSubset_total = find_testValSubset(0,noCvd_val,0,Cvd_val)
            print(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
            trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset1(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
    
    #creating X_train, y_train, X_test, y_test
        #για να δημιουργήσω το X_test αφαιρώ απλά την τελευταία στήλη
        X_test.append(np.delete(testSubset_total, 11, axis=1))
        y_test_temp = np.delete(testSubset_total, slice(0, 11), axis=1)
        #για το y_test κρατάω μόνο την τελευταία στήλη 
        y_test.append(np.reshape(y_test_temp, len(y_test_temp)))
        #οι ακόλουθες γραμμές κώδικα είναι για να φέρω τα X_test και y_test στην μορφή πίνακα που θέλω
        X_test_temp = np.array(X_test)
        X_test = X_test_temp[0]
        y_test_temp = np.array(y_test)
        y_test = y_test_temp[0]
        
        #για να δημιουργήσω το X_val αφαιρώ απλά την τελευταία στήλη
        X_val.append(np.delete(validationSubset_total, 11, axis=1))
        y_val_temp = np.delete(validationSubset_total, slice(0, 11), axis=1)
        #για το y_val κρατάω μόνο την τελευταία στήλη 
        y_val.append(np.reshape(y_val_temp, len(y_val_temp)))
        X_val_temp = np.array(X_val)
        X_val = X_val_temp[0]
        y_val_temp = np.array(y_val)
        y_val = y_val_temp[0]
        
        #για να δημιουργήσω το X_train αφαιρώ απλά την τελευταία στήλη
        X_train.append(np.delete(trainSubset_total, 11, axis=1))
        y_train_temp = np.delete(trainSubset_total, slice(0, 11), axis=1)
        #για το y_train κρατάω μόνο την τελευταία στήλη 
        y_train.append(np.reshape(y_train_temp, len(y_train_temp)))
        X_train_temp = np.array(X_train)
        X_train = X_train_temp[0]
        y_train_temp = np.array(y_train)
        y_train = y_train_temp[0]
        
    #αποθηκεύω τα X_test,y_test στον πίνακα test_total οπότε στην i-οστή θέση θα είναι τα X_test,y_test για το i fold 
    #το ίδιο κάνω και με τους πίνακες validation_total και train_total
        test_total.append([X_test,y_test])
        validation_total.append([X_val,y_val])
        train_total.append([X_train,y_train]) 
    #train_total_NoCvd:αποθηκεύω τα instances χωρίς Cvd πουθα χρησιμοποιηθούν για training όταν θα είμαστε στο i-fold
        train_total_NoCvd.append(trainSubset_NoCvd)
    #train_total_Cvd:αποθηκεύω τα instances με Cvd πουθα χρησιμοποιηθούν για training όταν θα είμαστε στο i-fold
        train_total_Cvd.append(trainSubset_Cvd)
    return train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd

In [31]:
train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd = create_train_test_sets(10,dataNoCvd,dataCvd,0.1)
# print((test_total[9][0]))

fold_Cvd_total  : 4  fold_Cvd_residue  : 1
fold_NoCvd_total: 51 fold_NoCvd_residue: 9
468 519 36 41
len 1-2: 468 36
len 3-4: 422 32
454


In [6]:
print(len(train_total[9][0]))

454


In [32]:
# Chech ratio in each train and test set and validation set
def find_ratio(index_list):
    one = 0
    lenght=len(index_list[0])
    for i in range(lenght):
#         print(index_list[1])
        if index_list[1][i] == 1.0 :
            one+= 1
    ratio = one/lenght
    return ratio

for i in range(10):
    print(len(train_total[i][0]))
    ratio = find_ratio(train_total[i])
    print("ratio in train set:", ratio)
    ratio = find_ratio(test_total[i])
    print("ratio in test set:", ratio)
    ratio = find_ratio(validation_total[i])
    print("ratio in validation set:", ratio)

454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in vali

In [66]:
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt

def my_fit(ratio,dataNoCvd,dataCvd, validation_total, params):   
    Subarray=[]
    models = []
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)

#Για ratio 1:2 πρέπει να φτιαχτούν 6 μοντέλα
    numOfSubsamples = rowsNoCvd//(rowsCvd*ratio)
    numOfSubsamples= int(numOfSubsamples)
    SubNoCvd = rowsNoCvd//numOfSubsamples
    residue = rowsNoCvd- SubNoCvd*numOfSubsamples


    Up = 0
    valid_preds = []

    for i in range(numOfSubsamples):
        classifier = XGBClassifier(base_score=params[11], booster='gbtree', colsample_bylevel=params[5],
                  colsample_bynode=params[12], colsample_bytree=params[8], gamma=params[4],
                  learning_rate=params[0], max_delta_step=params[13], max_depth=params[1],
                  min_child_weight=params[3], missing=None, n_estimators=params[2], n_jobs=1,
                  nthread=None, objective='binary:logistic', random_state=0,
                  reg_alpha=params[9], reg_lambda=params[6], scale_pos_weight=params[7], seed=None,
                  silent=None, subsample=params[10], verbosity=1)

#για να φτιάξουμε το training set αφού τα μοντέλα είναι 6 πρέπει να χωρίσουμε τα instances NoCvd στα 6
#στα πρώτα 9-fold τα NoCvd instances είναι 421, άρα 421//6=70 με υπόλοιπο 1
#στο 10ο fold τα NoCvd instances είναι 422, άρα 422//6=70 με υπόλοιπο 2
#τα υπόλοιπα μπαίνουν στα πρώτα μοντέλα δηλαδή στα πρώτα 9-fold το πρώτο μοντέλο θα πάρει 71 NoCvd instances
#και στο 10ο fold το πρώτο αλλά και το δεύτερο θα πάρουν 71 instances. Τα υπόλοιπα θα πάρουν από 70
        Down = Up
        Up= Up + SubNoCvd
        if i < residue :
            Up+= 1
        
        Sub1=dataNoCvd[Down:Up,:]
        #φτίαχνουμε πίνακα με τα NoCvd instances που βρήκαμε ότι αναλογούν σε κάθε μοντέλο και με όλα τα Cvd instances
        Sub2=np.concatenate((Sub1, dataCvd))
        Subarray.append(Sub2)
        
    #Dividing to X and y of the previous traing set    
        X=np.delete(Subarray[i], 11, axis=1)
        y=np.delete(Subarray[i], slice(0, 11), axis=1)
        y=np.reshape(y, len(y))
        
        X_val = validation_total[i][0]
        y_val = validation_total[i][1]
        
        eval_set  = [(X,y), (X_val,y_val)]
        #κάνουμε fit με χρήση early stopping
        classifier.fit( X, y, eval_set=eval_set, eval_metric="auc", early_stopping_rounds=50,verbose = False )
  
        print("model",i," :",classifier.best_iteration)
        
        #υπολογίζουμε το auc που έχει το μοντέλο για το validation set γιατί θα μας χρειαστεί στο weighted voting
        auc_valid = roc_auc_score(y_val, classifier.predict(X_val))
        print("auc_valid",auc_valid,classifier.best_score, classifier.best_ntree_limit)
        
        #αποθηκεύουμε τα auc σε πίνακα για να τα επιστέψουμε όλα μαζί
        valid_preds.append(auc_valid)

        #ο κώδικας που ακολουθεί και είναι σε σχόλια είναι για να τυπώνει το πρώτο δένδρο κάθε μοντέλου xgboost
#         plot_tree(classifier)     
#         plt.rcParams["figure.figsize"] = (40,7)
#         plot_tree(classifier, num_trees=0, rankdir='LR')
#         plt.show()

        models.append(classifier)
    return models,valid_preds

In [53]:
def voting(models,X,auc_valid):
    y_pred = []
    y_pred_models = []
    
    models_size = len(models)
    X_size = len(X)
            
    for i in range(models_size):
        y_pred_models.append(models[i].predict(X))
       
        
    for j in range(X_size):
        case = 0
        for i in range(models_size):
            if y_pred_models[i][j] == 1.0:
                case+=1
        if case >  models_size/2 :
            y_pred.append(1.) 
        else:
            y_pred.append(0.) 
            
    return y_pred  

In [54]:
def voting(models,X,weights):
#     print("efwef",len(weights), len(models))
    y_pred = []
    y_pred_models = []
    weights = [0.16667, 0.16667, 0.16667, 0.16667, 0.16667, 0.16667]
#     weights = [0.18, 0.16667, 0.16667, 0.16667, 0.16667, 0.16667]
#     weights = [0.15, 0.4, 0.2, 0.05, 0.05, 0.15]
#   weights = [0.16, 0.173, 0.2, 0.147, 0.16, 0.16]
    models_size = len(models)
#     print(models_size)
    X_size = len(X)
            
    for i in range(models_size):
        y_pred_models.append(models[i].predict_proba(X)[:,1]) 
        
    for j in range(X_size):
        sum = 0
        for i in range(models_size):
#             print(i)
#             print(y_pred_models[i][j])
#             print(weights[i])
            sum = sum + y_pred_models[i][j]*weights[i]
        avr = sum
#         /models_size
        if avr>= 0.5 :
            y_pred.append(1.)
        else:
            y_pred.append(0.)
    return y_pred   

In [55]:


def voting(models,X,weights):
    y_pred = []
    y_pred_models = []

    models_size = len(models)
    X_size = len(X)
       
    #βρίσκω και αποθηκευω τα predictions κάθε μοντέλου για το Χ_test όχι όμως με την μορφή (0,1) όπως θα έκανε η
# predict αλλά με μορφή πιθανότητας να έρθει 1 με την predict_proba, δηλαδή τώρα ο πίνακας y_pred_models έχει τις 
# πιθανότητες κάθε περίπτωση του X_test να εμφανίσει Cvd
    for i in range(models_size): 
        y_pred_models.append(models[i].predict_proba(X)[:,1]) 

#Παίρνω τις πιθανότητες κάθε μοντέλου και για κάθε μία περίπτωση του X_test κάνω το average των πιθανοτήτων αυτών
# οπότε βγαίνει μια μέση πιθανότητα για κάθε μία περίπτωση του X_test.
    for j in range(X_size):
        sum = 0
        for i in range(models_size):
            sum = sum + y_pred_models[i][j]
        avr = sum/models_size
        if avr>= 0.5 :         #Όμως επειδή θεωρώ ότι το μοντέλο κάνει classification αν η μέση πιθανότητα 
            y_pred.append(1.)  # είναι μεγαλυτερη από 0.5 τότε θα επιστρέφει 1, δηλαδή ότι έχει cvd, αλλιώς 0
        else:
            y_pred.append(0.)
    return y_pred   

In [56]:
#weighted voting
def weighted_voting(models,X,auc_valid):
    y_pred = []
    y_pred_models = []
    sum_auc = 0
    print("auc_valid:",mean(auc_valid))
    models_size = len(models)
    X_size = len(X)
            
    for i in range(models_size):
        y_pred_models.append(models[i].predict_proba(X)[:,1]) 
        sum_auc += auc_valid[i]
       
    for j in range(X_size):
        case = 0
        for i in range(models_size):
            case += y_pred_models[i][j]*(auc_valid[i]/sum_auc)*100 #auc_valid[i]/sum_auc --> η δύναμη της ψήφου
                                                                    #του μοντέλου

#έβαλα case>56 και όχι case>50(επειδή έχω κάνει *100 πίο πάνω μου ήταν πιο εύκολο στην κατανόηση, αλλίως
#θα έπρεπε να είναι case>0.5), διότι το case>56 έδινε καλύτερο τελικό auc              
        if case >  50 :
            y_pred.append(1.) 
        else:
            y_pred.append(0.) 
            
    return y_pred  

In [57]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6
#https://statinfer.com/204-4-2-calculating-sensitivity-and-specificity-in-python/

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from sklearn import metrics

def my_cross_val( cv, train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd,params):
    accuracy = []
    specificity = []
    sensitivity = []
    auc = []
    

    for i in range(cv):
        print("----------------",i,"- fold -----------------------")
        print("                  Estimators")
        #από τον πίνακα test_total παίρνω τα X_test,y_test για το i-fold
        X_test = test_total[i][0]
        y_test = test_total[i][1]
        
        #εφαρμόζω fit που μου επιστέφει το ensemble μοντέλο και τις επιδόσεις-auc του κάθε μοντέλου xgboost
        #που υπολογίζονται από τα validation set,από τα οποία αποτελείται το ensemble μοντέλο
        models,auc_valid = my_fit( 2, train_total_NoCvd[i], train_total_Cvd[i], validation_total,params)

        #εφαρμόζουμε weighted voting για να βρούμε την πρόβλεψη του ensemble model
        #στο weighted voting θα έχουν πιο ισχυρή ψήφο τα μοντέλα με μεγαλύτερο auc στο validation set 
        y_pred = weighted_voting(models,X_test,auc_valid)
#         y_pred = voting(models,X_test,auc_valid)

        cm = confusion_matrix(y_test, y_pred)

        total1=sum(sum(cm))
        #####from confusion matrix calculate accuracy
        accuracy1=(cm[0,0]+cm[1,1])/total1
        
#         accuracy1=accuracy_score(y_test, y_pred) #άλλος τρόπος για υπολογισμό accuracy

        specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])

        sensitivity1 = cm[1,1]/(cm[1,0]+cm[1,1])
        
        auc1 = roc_auc_score(y_test, y_pred)
        print(auc1)
        accuracy.append(accuracy1)
        specificity.append(specificity1)
        sensitivity.append(sensitivity1) 
        auc.append(auc1)
    #επιστρέφουμε πίνακες και με τα 10 accuracy, specificity, sensitivity, auc που προκύπτουν από το 10 fold cross
    #validation
    print(auc)
    return accuracy, specificity, sensitivity, auc

In [58]:
# https://www.geeksforgeeks.org/python-all-possible-permutations-of-n-lists/

def compute_comb(params):
    
    # initializing lists 
    list1 = params.get("learning_rate")
    list2 = params.get("max_depth")
    list3 = params.get("n_estimators") 
    list4 = params.get("min_child_weight")
    list5 = params.get("gamma")    
    list6 = params.get("colsample_bylevel")
    list7 = params.get("reg_lambda") 
    list8 = params.get("scale_pos_weight")
    list9 = params.get("colsample_bytree")
    list10 = params.get("reg_alpha")
    list11 = params.get("subsample")
    list12 = params.get("base_score")
    list13 = params.get("colsample_bynode")
    list14 = params.get("max_delta_step")
    
    # to compute all possible permutations 
    res = [[i, j, k, l, m, n, o, p,q,r,s,t, u, v] for i in list1  
                                                  for j in list2 
                                                  for k in list3 
                                                  for l in list4 
                                                  for m in list5 
                                                  for n in list6 
                                                  for o in list7
                                                  for p in list8
                                                  for q in list9
                                                  for r in list10
                                                  for s in list11
                                                  for t in list12
                                                  for u in list13
                                                  for v in list14] 

    return res

In [59]:
class style:
    BOLD = '\033[1m'
    END = '\033[0m'

In [60]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from statistics import mean

def gridsearch(params):
    max_sens = 0
    max_spes = 0
    max_acc = 0
    best_params = []
    max_params = []
    max_auc = 0
    #compute_comb = υπολογίζει όλους τους συνδυασμους των παραμέτρων που της δώσαμε να ελέγξει
    combs = compute_comb(params)
    print(len(combs))
    
    #για κάθε έναν συνδυασμό θα κάνουμε cross validation
    for j in range(len(combs)):
        print(j)
        accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd,combs[j])
        if mean(auc)>0.50:
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
            print(combs[j])
            print (style.BOLD + 'accuracy    ' + style.END, mean(accuracy))
            print (style.BOLD + 'specificity ' + style.END, mean(specificity))
            print (style.BOLD + 'sensitivity ' + style.END, mean(sensitivity))
            print (style.BOLD + 'auc         ' + style.END, mean(auc))
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
        
        #ο πίνακας best_params θα περιέχει όλα τα αποτελέσματα όλων των συνδυασμών για να μπορούμε στο τέλος
        #της gridsearch να βρούμε τα καλύτερα
        #βλέπουμε ότι γίνεται αποθήκευση των mean γιατί η my_cross_val πιστρέφουμε πίνακες και με τα 
        #10 accuracy, specificity, sensitivity, auc που προκύπτουν από το 10 fold cross validation
        best_params.append([combs[j],mean(accuracy),mean(specificity),mean(sensitivity),mean(auc)])
        #αποθηκεύουμε τον συνδυασμό των παραμέτρων που μας δίνει το καλύτερο auc
        if mean(auc)> max_auc:
            max_sens = mean(sensitivity)
            max_spes = mean(specificity)
            max_acc = mean(accuracy)
            max_auc = mean(auc)
            max_params = combs[j]
    return max_sens, max_params, max_spes, max_acc, max_auc, best_params

# Ratio = 2

In [17]:
params = {
                    "learning_rate"    :[0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[3],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1 0.75
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[4],
                    "colsample_bytree" :[0.75],
                    "reg_alpha"        :[0],
                    "subsample"        :[1.0],
                    "base_score"       :[0.5],
                    "colsample_bynode" :[1],
                    "max_delta_step"   :[0]
            
        }
parameters : [0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy : 0.6053571428571429
spes     : 0.5898567119155355
sens     : 0.81
auc      : 0.6999283559577677
[3, 15, 13, 14, 11, 2, 1, 12, 10, 5, 6, 0, 8]


[3, 15, 13, 14, 11, 2, 1, 12, 10, 5, 6, 0, 8]

In [26]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

#50 = early stopping


1
0
---------------- 0 - fold -----------------------
                  Estimators
model 0  : 43
auc_valid 0.7010869565217391 0.630435 44
model 1  : 40
auc_valid 0.8478260869565217 0.978261 41
model 2  : 169
auc_valid 0.8369565217391304 0.945652 170
model 3  : 63
auc_valid 0.7391304347826086 0.88587 64
model 4  : 41
auc_valid 0.7391304347826086 0.945652 42
model 5  : 59
auc_valid 0.7608695652173914 0.896739 60
0.6923076923076922
---------------- 1 - fold -----------------------
                  Estimators
model 0  : 13
auc_valid 0.625 0.663043 14
model 1  : 50
auc_valid 0.5054347826086957 0.554348 51
model 2  : 84
auc_valid 0.8478260869565217 1.0 85
model 3  : 54
auc_valid 0.7173913043478262 0.913043 55
model 4  : 82
auc_valid 0.7717391304347826 0.961957 83
model 5  : 2
auc_valid 0.6956521739130435 0.869565 3
0.6826923076923078
---------------- 2 - fold -----------------------
                  Estimators
model 0  : 65
auc_valid 0.8804347826086957 0.945652 66
model 1  : 12
auc_valid 0

In [31]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)
#v=56
#50 = early stopping

1
0
---------------- 0 - fold -----------------------
                  Estimators
model 0  : 43
auc_valid 0.7010869565217391 0.630435 44
model 1  : 40
auc_valid 0.8478260869565217 0.978261 41
model 2  : 169
auc_valid 0.8369565217391304 0.945652 170
model 3  : 63
auc_valid 0.7391304347826086 0.88587 64
model 4  : 41
auc_valid 0.7391304347826086 0.945652 42
model 5  : 59
auc_valid 0.7608695652173914 0.896739 60
auc_valid: 0.7708333333333333
0.75
---------------- 1 - fold -----------------------
                  Estimators
model 0  : 13
auc_valid 0.625 0.663043 14
model 1  : 50
auc_valid 0.5054347826086957 0.554348 51
model 2  : 84
auc_valid 0.8478260869565217 1.0 85
model 3  : 54
auc_valid 0.7173913043478262 0.913043 55
model 4  : 82
auc_valid 0.7717391304347826 0.961957 83
model 5  : 2
auc_valid 0.6956521739130435 0.869565 3
auc_valid: 0.6938405797101449
0.7307692307692307
---------------- 2 - fold -----------------------
                  Estimators
model 0  : 65
auc_valid 0.88043478

In [49]:
params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[4],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0.5],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1 0.75
                     "reg_lambda"      :[2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.5],
                    "reg_alpha"        :[0],
                    "subsample"        :[1.0],
                    "base_score"       :[0.5],
                    "colsample_bynode" :[1],
                    "max_delta_step"   :[0]
            
        }
# parameters : [0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
# accuracy : 0.6285714285714286
# spes     : 0.616817496229261
# sens     : 0.785
# auc      : 0.7009087481146304
# [14, 15, 3, 13, 11, 1, 2, 12, 5, 10, 8]
# [14, 15, 3, 13, 11, 1, 2, 12, 5, 10, 8, 0, 4, 6, 7, 9, 16]
# 14-15-3-13-11-1-2-12-5-10-8
# plin-0-4-6-7-9

In [67]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

1
0
---------------- 0 - fold -----------------------
                  Estimators
model 0  : 7
auc_valid 0.6467391304347825 0.720109 8
model 1  : 35
auc_valid 0.891304347826087 0.994565 36
model 2  : 37
auc_valid 0.8586956521739131 0.967391 38
model 3  : 74
auc_valid 0.7391304347826086 0.896739 75
model 4  : 73
auc_valid 0.7608695652173914 0.961957 74
model 5  : 4
auc_valid 0.7173913043478262 0.918478 5
auc_valid: 0.7690217391304348
0.8557692307692308
---------------- 1 - fold -----------------------
                  Estimators
model 0  : 8
auc_valid 0.6902173913043479 0.75 9
model 1  : 14
auc_valid 0.6304347826086957 0.592391 15
model 2  : 56
auc_valid 0.8804347826086957 0.967391 57
model 3  : 38
auc_valid 0.7065217391304348 0.891304 39
model 4  : 65
auc_valid 0.7717391304347826 0.951087 66
model 5  : 5
auc_valid 0.7391304347826086 0.918478 6
auc_valid: 0.7364130434782609
0.7596153846153846
---------------- 2 - fold -----------------------
                  Estimators
model 0  : 49
